# Table of contents
* [Read baseline with GDP and population](#reading-gdp-and-population)
* [Reading emissions](#reading-initial-emissions)
* [Calculating shares for effort sharing](#calculate-shares)
    * [GDP per capita](#average-gdp-2015-2100)
    * [Per capita](#average-population-2015-2100)
    * [Emission shares](#emissions-shares)
* [Merge all the approaches](#all-approaches)
* [Save csv files of emission shares](#export)
* [Get .txt of batch commands to run model](#batch-commands)

In [2]:
from gams.transfer import Container
import pandas as pd
import numpy as np
from utils import read_gdx,read_scenarios

# Reading GDP and population

In [9]:
cols = ['region_def','ssp','t','year','n','pop(mill)'] #names to be used for columns
data = pd.DataFrame(columns=cols)
region_defs = ['r5','witch17','ed57']

for region_def in region_defs:
    path = f'data_{region_def}/data_baseline.gdx'
    gdx = Container(path) #values for ssp_l (population), ssp_ykali (GDP_MER), ppp2mer
    reg_data = gdx['ssp_l'].records.rename(columns={'uni_0':'ssp','t_1':'t','n_2':'n','value':'pop(mill)'}) # Get population values and rename columns
    gdp = Container(path)['ssp_ykali'].records['value']
    reg_data['gdp_mer'] = gdp

    gdx = Container(path)
    gdx1 = gdx['ppp2mer'].records
    gdx2 = gdx1.drop(columns='t_0').drop_duplicates().rename(columns={'value':'ppp2mer','n_1':'n'})
    gdx2['mer2ppp'] = 1/gdx2['ppp2mer']

    reg_data= pd.merge(reg_data,gdx2,'left','n')
    reg_data['gdp_ppp (trill USD)'] = reg_data['gdp_mer']*reg_data['mer2ppp']
    reg_data['gdp_percapita (USD)'] = reg_data['gdp_ppp (trill USD)']/reg_data['pop(mill)']*1000000
    
    reg_data['t'] = reg_data['t'].astype(int) #t as integer
    reg_data['year'] = 2010 + 5*reg_data['t'] #get year column
    reg_data['region_def'] = region_def #region definition (r5,witch17,ed57)
    data = pd.concat([data,reg_data])

#data = data[cols] #re-organize columns
data 

,region_def,ssp,t,year,n,pop(mill),gdp_mer,ppp2mer,mer2ppp,gdp_ppp (trill USD),gdp_percapita (USD)
0,r5,SSP1,1,2015,r5asia,3760.084,10.172933,0.413535,2.418177,24.599949,6542.393444
1,r5,SSP1,1,2015,r5lam,615.346,3.977246,0.571023,1.751244,6.965130,11319.046627
2,r5,SSP1,1,2015,r5maf,1314.522,3.069378,0.499862,2.000553,6.140453,4671.244049
3,r5,SSP1,1,2015,r5oecd,1037.214,38.914751,1.052591,0.950037,36.970446,35643.990837
4,r5,SSP1,1,2015,r5ref,483.682,3.411454,0.504863,1.980734,6.757183,13970.299816
...,...,...,...,...,...,...,...,...,...,...,...
16525,ed57,SSP5,9,2055,tur,86.588,2.943893,0.618090,1.617887,4.762887,55006.317273
16526,ed57,SSP5,9,2055,ukr,38.647,0.615413,0.327680,3.051758,1.878092,48596.061790
16527,ed57,SSP5,9,2055,usa,501.112,45.165300,1.000000,1.000000,45.165300,90130.150545
16528,ed57,SSP5,9,2055,vnm,94.396,0.913959,0.297240,3.364285,3.074820,32573.626001


# Reading initial emissions

In [10]:
emissions = pd.DataFrame()

for region_def in region_defs:
    path = f'data_{region_def}/data_baseline_emissions_calibrated.gdx'

    gdx = Container(path)
    emis_data = gdx['emi_bau_calibrated'].records.rename(columns={'uni_0':'ssp','t_1':'t','n_2':'n','value':'eind_0'})
    emis_data = emis_data[emis_data['t'] == '1']


    path1 = f'data_{region_def}/data_historical_values.gdx'

    gdx = Container(path1)
    emis_land = gdx['q_emi_valid_primap'].records.rename(columns={'uni_1':'t','n_2':'n','value':'eland'})
    emis_land = emis_land[(emis_land['t'] <= '2015') & (emis_land['t'] >= '2005') & (emis_land['uni_0'] == 'co2lu')]
    emis_land['eland'] = emis_land['eland']*3.66667 #Gton CO2
    eland_avg = emis_land.groupby('n')['eland'].mean().reset_index()

    tot_emis = pd.merge(emis_data,eland_avg,'left','n')
    tot_emis = tot_emis.rename(columns={'eland_y':'eland'})

    tot_emis['e0'] = tot_emis['eind_0'] + tot_emis['eland']
    tot_emis['region_def'] = region_def #region definition (r5,witch17,ed57)
    emissions = pd.concat([emissions,tot_emis])

emissions = emissions.drop(columns='t')
emissions['ssp'] = emissions['ssp'].str.upper()

In [156]:
emissions

,ssp,n,eind_0,eland,e0,region_def
0,SSP1,r5asia,14.329429,1.387788,15.717217,r5
1,SSP1,r5lam,1.903000,1.135689,3.038689,r5
2,SSP1,r5maf,2.994176,1.388626,4.382802,r5
3,SSP1,r5oecd,11.736576,-0.472147,11.264429,r5
4,SSP1,r5ref,3.582978,-0.423256,3.159722,r5
...,...,...,...,...,...,...
280,SSP5,tur,0.380229,-0.056400,0.323829,ed57
281,SSP5,ukr,0.223654,-0.014336,0.209317,ed57
282,SSP5,usa,5.505983,-0.231960,5.274023,ed57
283,SSP5,vnm,0.203652,-0.000328,0.203324,ed57


# Calculate shares

## Average GDP 2015-2100

In [11]:
ssps = ['SSP1','SSP2','SSP3','SSP4','SSP5'] #List of all ssps
gdp_avg = pd.DataFrame()

for region_def in region_defs:
    for ssp in ssps:
        gdp_data = data[(data['region_def'] == region_def) & (data['year'] <= 2100) & (data['ssp'] == ssp)]
        gdp_data = gdp_data.groupby(['n']).agg({'gdp_percapita (USD)':['mean']}).reset_index()
        gdp_data['gdp_share_pc'] = gdp_data['gdp_percapita (USD)'] / gdp_data['gdp_percapita (USD)'].sum() # % of projected population share

        gdp_data['gdp_percapita_inv'] = 1 / gdp_data['gdp_percapita (USD)']
        gdp_data['gdp_share_pc_inv'] = gdp_data['gdp_percapita_inv'] / gdp_data['gdp_percapita_inv'].sum()
        
        gdp_data[['region_def','ssp']] = [region_def,ssp] #columns with region
        gdp_avg = pd.concat([gdp_avg,gdp_data])

gdp_avg.columns = gdp_avg.columns.droplevel(1)
gdp_avg

,n,gdp_percapita (USD),gdp_share_pc,gdp_percapita_inv,gdp_share_pc_inv,region_def,ssp
0,r5asia,37432.581489,0.172353,0.000027,0.210538,r5,SSP1
1,r5lam,41777.330223,0.192358,0.000024,0.188642,r5,SSP1
2,r5maf,25466.666635,0.117258,0.000039,0.309462,r5,SSP1
3,r5oecd,67267.228421,0.309723,0.000015,0.117159,r5,SSP1
4,r5ref,45241.282983,0.208307,0.000022,0.174199,r5,SSP1
...,...,...,...,...,...,...,...
52,tur,64586.452814,0.016422,0.000015,0.017490,ed57,SSP5
53,ukr,59069.090644,0.015019,0.000017,0.019123,ed57,SSP5
54,usa,98281.238912,0.024989,0.000010,0.011493,ed57,SSP5
55,vnm,43012.668627,0.010936,0.000023,0.026262,ed57,SSP5


## Average population 2015-2100

In [12]:
pop_shares = pd.DataFrame()

# For every region_def and ssp compute the avg population until 2100 and the corresponding pop_share
for region_def in region_defs:
    for ssp in ssps:
        new_data = data[(data['region_def'] == region_def) & (data['year'] <= 2100) & (data['ssp'] == ssp)] # fitler for region, ssp and years
        new_data = new_data.groupby(['n']).agg({'pop(mill)':['mean']}).reset_index() #group by region and compute mean population
        new_data['pop_share'] = new_data['pop(mill)'] / new_data['pop(mill)'].sum() # % of projected population share
        new_data[['region_def','ssp']] = [region_def,ssp] #columns with region
        pop_shares = pd.concat([pop_shares,new_data])

cols = ['region_def','ssp','n','avg_pop(mill)','pop_share']
pop_shares.columns = pop_shares.columns.droplevel(1)
pop_shares = pop_shares.rename(columns={'pop(mill)':'avg_pop(mill)'})[cols]
pop_shares


,region_def,ssp,n,avg_pop(mill),pop_share
0,r5,SSP1,r5asia,3701.456444,0.467290
1,r5,SSP1,r5lam,622.102333,0.078537
2,r5,SSP1,r5maf,1977.885889,0.249698
3,r5,SSP1,r5oecd,1181.765722,0.149192
4,r5,SSP1,r5ref,437.894389,0.055282
...,...,...,...,...,...
52,ed57,SSP5,tur,80.489000,0.009935
53,ed57,SSP5,ukr,37.178444,0.004589
54,ed57,SSP5,usa,518.115667,0.063955
55,ed57,SSP5,vnm,86.768000,0.010711


## Current emissions shares

In [13]:
emi_shares = pd.DataFrame()

for region_def in region_defs:
    for ssp in ssps:
        new_data = emissions[(emissions['ssp'] == ssp) & (emissions['region_def'] == region_def)]
        new_data[['region_def','ssp']] = [region_def,ssp]

        new_data = pd.merge(new_data,data[data['year'] == 2015],'left',['region_def','ssp','n'])
        new_data['emi_per_capita'] = new_data['e0']/new_data['pop(mill)'] #ton/person
        new_data['emi_shares_pc'] = new_data['emi_per_capita'] / new_data['emi_per_capita'].sum()

        new_data['e0_inv'] = 1 / new_data['emi_per_capita']
        new_data['emi_shares_pc_inv'] = new_data['e0_inv'] / new_data['e0_inv'].sum()

        emi_shares = pd.concat([emi_shares,new_data])

C:\Users\danip\AppData\Local\Temp\ipykernel_9352\3865417627.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data[['region_def','ssp']] = [region_def,ssp]
C:\Users\danip\AppData\Local\Temp\ipykernel_9352\3865417627.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data[['region_def','ssp']] = [region_def,ssp]
C:\Users\danip\AppData\Local\Temp\ipykernel_9352\3865417627.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

## Historial emissions

In [27]:
hist_emis = pd.read_csv('owid-co2-data.csv',usecols=['country','year','iso_code','cumulative_co2','population'])
hist_emis = hist_emis[hist_emis['year'] == 2015]

map_codes = pd.read_csv('map_codes.csv',delimiter=';')
map_codes = pd.merge(map_codes,hist_emis[['iso_code','cumulative_co2','population']],'left',left_on='ISO',right_on='iso_code')
map_codes['cum_emis_percapita'] = map_codes['cumulative_co2'] / map_codes['population']
map_codes

,ISO,ed57,r5,witch17,iso_code,cumulative_co2,population,cum_emis_percapita
0,BES,rcam,r5lam,laca,BES,8.016,23193.0,0.000346
1,CUW,rcam,r5lam,laca,CUW,555.769,169588.0,0.003277
2,SXM,rcam,r5lam,laca,SXM,66.225,40228.0,0.001646
3,ESH,noan,r5maf,mena,ESH,NaN,491837.0,NaN
4,AUT,aut,r5oecd,europe,AUT,5173.740,8642422.0,0.000599
...,...,...,...,...,...,...,...,...
245,KGZ,ris,r5ref,te,KGZ,791.669,5914985.0,0.000134
246,MDA,ris,r5ref,te,MDA,1309.822,3277390.0,0.000400
247,TJK,ris,r5ref,te,TJK,434.565,8524066.0,0.000051
248,TKM,ris,r5ref,te,TKM,2051.231,5766427.0,0.000356


In [37]:
total_hist_emis = pd.DataFrame()
for region_def in region_defs:
    reg_emis = map_codes[[region_def,'ISO','cumulative_co2','population']]
    reg_emis = reg_emis.groupby(region_def)['cumulative_co2','population'].sum().reset_index()
    reg_emis['cumemis_percapita'] = reg_emis['cumulative_co2'] / reg_emis['population'] * 1000000 #ton/person

    reg_emis['region_def'] = region_def
    reg_emis = reg_emis.rename(columns={region_def:'n'})
    reg_emis['log10_cumemis'] = np.log10(reg_emis['cumulative_co2'])
    reg_emis['hist_emi_share'] = reg_emis['log10_cumemis'] / reg_emis['log10_cumemis'].sum()

    reg_emis['cum_co2_inv'] = 1 / reg_emis['log10_cumemis']
    reg_emis['hist_emi_share_inv'] = reg_emis['cum_co2_inv'] / reg_emis['cum_co2_inv'].sum()
    total_hist_emis = pd.concat([total_hist_emis,reg_emis])

total_hist_emis

C:\Users\danip\AppData\Local\Temp\ipykernel_9352\266414321.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  reg_emis = reg_emis.groupby(region_def)['cumulative_co2','population'].sum().reset_index()
C:\Users\danip\AppData\Local\Temp\ipykernel_9352\266414321.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  reg_emis = reg_emis.groupby(region_def)['cumulative_co2','population'].sum().reset_index()
C:\Users\danip\AppData\Local\Temp\ipykernel_9352\266414321.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  reg_emis = reg_emis.groupby(region_def)['cumulative_co2','population'].sum().reset_index()


,n,cumulative_co2,population,cumemis_percapita,region_def,log10_cumemis,hist_emi_share,cum_co2_inv,hist_emi_share_inv
0,r5asia,279200.230,3.876167e+09,72.029981,r5,5.445916,0.206386,0.183624,0.192608
1,r5lam,62199.408,6.230696e+08,99.827391,r5,4.793786,0.181672,0.208603,0.218810
2,r5maf,69332.762,1.394432e+09,49.721161,r5,4.840939,0.183459,0.206572,0.216678
3,r5oecd,827170.682,1.039315e+09,795.880566,r5,5.917595,0.224261,0.168988,0.177256
4,r5ref,244808.724,4.902583e+08,499.346411,r5,5.388827,0.204222,0.185569,0.194648
...,...,...,...,...,...,...,...,...,...
52,tur,8821.333,7.964618e+07,110.756516,ed57,3.945534,0.017348,0.253451,0.017420
53,ukr,29465.105,4.498257e+07,655.033857,ed57,4.469308,0.019651,0.223748,0.015378
54,usa,391085.312,3.246078e+08,1204.793418,ed57,5.592272,0.024588,0.178818,0.012290
55,vnm,2894.583,9.219140e+07,31.397538,ed57,3.461586,0.015220,0.288885,0.019855


In [32]:
region_def = 'r5'
reg_emis = map_codes[[region_def,'ISO','cumulative_co2']]

avg_emis = reg_emis.groupby(region_def)['cumulative_co2'].mean()
avg_emis

r5
r5asia     7755.561944
r5lam      1382.209067
r5maf       990.468029
r5oecd    23633.448057
r5ref      7897.055613
Name: cumulative_co2, dtype: float64

In [45]:
# Fill missing values
missing_values = reg_emis['cumulative_co2'].isna()
reg_missing = reg_emis[region_def][missing_values] #regions of missing values
new_values = avg_emis[reg_missing]
new_values

r5
r5maf       990.468029
r5oecd    23633.448057
r5oecd    23633.448057
r5oecd    23633.448057
r5oecd    23633.448057
r5oecd    23633.448057
r5oecd    23633.448057
r5oecd    23633.448057
r5oecd    23633.448057
r5oecd    23633.448057
r5ref      7897.055613
r5maf       990.468029
r5maf       990.468029
r5maf       990.468029
r5oecd    23633.448057
r5oecd    23633.448057
r5oecd    23633.448057
r5oecd    23633.448057
r5oecd    23633.448057
r5oecd    23633.448057
r5asia     7755.561944
r5asia     7755.561944
r5asia     7755.561944
r5asia     7755.561944
r5asia     7755.561944
r5lam      1382.209067
r5lam      1382.209067
r5lam      1382.209067
r5lam      1382.209067
r5lam      1382.209067
r5lam      1382.209067
r5lam      1382.209067
r5lam      1382.209067
Name: cumulative_co2, dtype: float64

In [48]:
reg_emis['cumulative_co2'][missing_values] = new_values

C:\Users\danip\AppData\Local\Temp\ipykernel_3120\1671159532.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reg_emis['cumulative_co2'][missing_values] = new_values
C:\Users\danip\AppData\Local\Temp\ipykernel_3120\1671159532.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reg_emis['cumulative_co2'][missing_values] = new_values


# All approaches

In [38]:
whole_data = pd.merge(pop_shares,gdp_avg,'left',['region_def','n','ssp'])
whole_data = pd.merge(whole_data,emi_shares,'left',['region_def','n','ssp'])
whole_data = pd.merge(whole_data,total_hist_emis,'left',['region_def','n'])
shares = whole_data[['region_def','ssp','n','pop_share','gdp_share_pc','gdp_share_pc_inv','emi_shares_pc','emi_shares_pc_inv','hist_emi_share','hist_emi_share_inv']]

In [39]:
shares

,region_def,ssp,n,pop_share,gdp_share_pc,gdp_share_pc_inv,emi_shares_pc,emi_shares_pc_inv,hist_emi_share,hist_emi_share_inv
0,r5,SSP1,r5asia,0.467290,0.172353,0.210538,0.140056,0.242428,0.206386,0.192608
1,r5,SSP1,r5lam,0.078537,0.192358,0.188642,0.165459,0.205208,0.181672,0.218810
2,r5,SSP1,r5maf,0.249698,0.117258,0.309462,0.111714,0.303933,0.183459,0.216678
3,r5,SSP1,r5oecd,0.149192,0.309723,0.117159,0.363886,0.093308,0.224261,0.177256
4,r5,SSP1,r5ref,0.055282,0.208307,0.174199,0.218884,0.155122,0.204222,0.194648
...,...,...,...,...,...,...,...,...,...,...
390,ed57,SSP5,tur,0.009935,0.016422,0.017490,0.012222,0.015996,0.017348,0.017420
391,ed57,SSP5,ukr,0.004589,0.015019,0.019123,0.013726,0.014243,0.019651,0.015378
392,ed57,SSP5,usa,0.063955,0.024989,0.011493,0.046904,0.004168,0.024588,0.012290
393,ed57,SSP5,vnm,0.010711,0.010936,0.026262,0.006395,0.030572,0.015220,0.019855


In [40]:
effort_shares = pd.melt(shares,id_vars=['region_def','ssp','n'],var_name='approach',value_name='budget_share')
effort_shares.replace({'pop_share':'percapita',
                       'gdp_share_pc':'gdp_percapita',
                       'gdp_share_pc_inv': 'gdp_percapita_inv',
                       'emi_shares_pc': 'current_emis_percapita',
                       'emi_shares_pc_inv':'current_emis_percapita_inv',
                       'hist_emi_share':'hist_emis',
                       'hist_emi_share_inv':'hist_emis_inv'},
                       inplace=True)
effort_shares = effort_shares[['region_def','ssp','approach','n','budget_share']]

In [41]:
effort_shares

,region_def,ssp,approach,n,budget_share
0,r5,SSP1,percapita,r5asia,0.467290
1,r5,SSP1,percapita,r5lam,0.078537
2,r5,SSP1,percapita,r5maf,0.249698
3,r5,SSP1,percapita,r5oecd,0.149192
4,r5,SSP1,percapita,r5ref,0.055282
...,...,...,...,...,...
2760,ed57,SSP5,hist_emis_inv,tur,0.017420
2761,ed57,SSP5,hist_emis_inv,ukr,0.015378
2762,ed57,SSP5,hist_emis_inv,usa,0.012290
2763,ed57,SSP5,hist_emis_inv,vnm,0.019855


# Export

In [42]:
for reg_def in region_defs:
    new_data = effort_shares[effort_shares['region_def'] == reg_def]
    new_data.to_csv(f'data_{reg_def}/effort_shares.csv',index=False)

effort_shares.to_csv('all_shares.csv',index=False)

Global cost of mitigation
GINI index

# Batch commands

In [66]:
# Baseline scenarios
bau_coop = 'gams run_rice50x.gms --policy=bau --impact=off --cooperation=coop --nameout=bau_coop'
bau_noncoop = 'gams run_rice50x.gms --policy=bau --impact=off --cooperation=noncoop --nameout=bau_noncoop'
commands_baseline = [bau_coop,bau_noncoop]

In [75]:
# Gamma scenarios
ssps = ['ssp1','ssp2','ssp3','ssp4','ssp5']
gammas = [0,0.5,0.8,1.2,1.5]
budgets = [750,1000,1250,1500]
max_miuup = 2.5

commands_gamma = []
for ssp in ssps:
    for cbudget in budgets:
        for gamma in gammas:
            text = f'gams run_rice50x.gms --impact=off --cooperation=coop --cea-cbudget --sharing=no --t_min_miu=7 --t_max_miu=17 --max_miuup={max_miuup} --cbudget={cbudget} --baseline={ssp} --gamma={gamma} --nameout=gamma{gamma}_{ssp}_{cbudget}'
            commands_gamma.append(text)

In [76]:
# Commands effort sharing
approaches = ['percapita','hist_emis_inv','gdp_percapita_inv','current_emis_percapita','hist_emis','gdp_percapita','current_emis_percapita_inv']

commands_sharing = []
for ssp in ssps:
    for cbudget in budgets:
        for approach in approaches:
            text=f'gams run_rice50x.gms --impact=off --cooperation=coop --cea-cbudget --sharing=yes --t_min_miu=7 --t_max_miu=17 --max_miuup={max_miuup} --cbudget={cbudget} --baseline={ssp} --approach={approach} --nameout=approach_{approach}_{ssp}_{cbudget}'
            commands_sharing.append(text)

In [77]:
all_commands = commands_baseline + commands_gamma + commands_sharing
all_commands[0:10]


['gams run_rice50x.gms --policy=bau --impact=off --cooperation=coop --nameout=bau_coop',
 'gams run_rice50x.gms --policy=bau --impact=off --cooperation=noncoop --nameout=bau_noncoop',
 'gams run_rice50x.gms --impact=off --cooperation=coop --cea-cbudget --sharing=no --t_min_miu=7 --t_max_miu=17 --max_miuup=2.5 --cbudget=750 --baseline=ssp1 --gamma=0 --nameout=gamma0_ssp1_750',
 'gams run_rice50x.gms --impact=off --cooperation=coop --cea-cbudget --sharing=no --t_min_miu=7 --t_max_miu=17 --max_miuup=2.5 --cbudget=750 --baseline=ssp1 --gamma=0.5 --nameout=gamma0.5_ssp1_750',
 'gams run_rice50x.gms --impact=off --cooperation=coop --cea-cbudget --sharing=no --t_min_miu=7 --t_max_miu=17 --max_miuup=2.5 --cbudget=750 --baseline=ssp1 --gamma=0.8 --nameout=gamma0.8_ssp1_750',
 'gams run_rice50x.gms --impact=off --cooperation=coop --cea-cbudget --sharing=no --t_min_miu=7 --t_max_miu=17 --max_miuup=2.5 --cbudget=750 --baseline=ssp1 --gamma=1.2 --nameout=gamma1.2_ssp1_750',
 'gams run_rice50x.gms -

In [78]:
# Write output txt
with open('all_commands.txt','w') as file:
    for line in all_commands:
        file.write(line+'\n')